In [1]:
!pip install -qq google-play-scraper
!pip install -qq -U watermark

%reload_ext watermark
%watermark -v -p pandas,matplotlib,seaborn,google_play_scraper


Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.14.0

pandas             : 2.0.2
matplotlib         : 3.8.0
seaborn            : 0.13.0
google_play_scraper: 1.2.4



In [2]:

import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)



In [3]:
app_packages = ['com.openai.chatgpt']

In [4]:
app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en')
  del info['comments']
  app_infos.append(info)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.57it/s]


In [5]:
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [6]:
print_json(app_infos[0])

{
  "adSupported": false,
  "appId": "com.openai.chatgpt",
  "categories": [
    {
      "id": "PRODUCTIVITY",
      "name": "Productivity"
    }
  ],
  "containsAds": false,
  "contentRating": "Teen",
  "contentRatingDescription": "Diverse Content: Discretion Advised",
  "currency": "USD",
  "description": "OpenAI\u2019s latest advancements at your fingertips.\r\n\r\nThis official app is free, syncs your history across devices, and brings you the newest model improvements from OpenAI.\r\n\r\nWith ChatGPT in your pocket, you\u2019ll find:\r\n\r\n\u00b7 Instant answers\r\n\u00b7 Tailored advice\r\n\u00b7 Creative inspiration\r\n\u00b7 Professional input\r\n\u00b7 Learning opportunities\r\n\r\nJoin millions of users and try out the app that\u2019s been captivating the world. Download ChatGPT today.\r\n\r\nTerms of service & privacy policy:\r\nhttps://openai.com/policies/terms-of-use\r\nhttps://openai.com/policies/privacy-policy",
  "descriptionHTML": "OpenAI\u2019s latest advancements at

In [7]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv ('com.openai.chatgpt', index=None, header=True)

In [15]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        #country='us',
        sort=sort_order,
        count= 20000,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [05:53<00:00, 353.50s/it]


In [16]:
print_json(app_reviews[0])

{
  "appId": "com.openai.chatgpt",
  "appVersion": "1.2023.281",
  "at": "2023-10-18 17:15:49",
  "content": "The AI was casually racist against Arabic nations and spoke positively in support of terrorist organisations like the Irgun (despite them indiscriminately killing women and children). It's a very useful tool if you want to brainstorm something as a sounding board, but I would be hesitant to use such an incredibly biased tool for information retrieval. I'd have honestly given it a 4* review prior to today, as beyond some short-term memory issues and rigidity it was great. Not with racism.",
  "repliedAt": null,
  "replyContent": null,
  "reviewCreatedVersion": "1.2023.281",
  "reviewId": "ecf711c0-13bc-4493-95a6-4b97327a0d78",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 0,
  "userImage": "https://play-lh.googleusercontent.com/a/ACg8ocI5pZxC4sMQ5ky7kPhe1AzT0YTPxul8Cj_gUBGCmU0M=mo",
  "userName": "Tom Lord"
}



In [17]:
len(app_reviews)

55939

In [18]:
app_reviews_df = pd.DataFrame(app_reviews)
#app_infos_df.to_csv ('com.openai.chatgpt', index=None, header=True)

In [19]:
app_reviews_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sortOrder,appId
0,ecf711c0-13bc-4493-95a6-4b97327a0d78,Tom Lord,https://play-lh.googleusercontent.com/a/ACg8oc...,The AI was casually racist against Arabic nati...,1,0,1.2023.281,2023-10-18 17:15:49,None,NaT,1.2023.281,most_relevant,com.openai.chatgpt
1,1f4cfa28-6dd4-4e39-a088-866d07b7f882,José Pérez,https://play-lh.googleusercontent.com/a/ACg8oc...,"It stopped working, it loads for a little bit....",1,3,1.2023.281,2023-10-16 15:14:14,None,NaT,1.2023.281,most_relevant,com.openai.chatgpt
2,9d692923-faa8-4ff0-887b-4274a3ea38bf,Steve Gilbert,https://play-lh.googleusercontent.com/a-/ALV-U...,I've not been able to run this app so my revie...,1,5,1.2023.243,2023-09-16 22:32:22,None,NaT,1.2023.243,most_relevant,com.openai.chatgpt
3,8bd66c82-ff65-43ae-9a17-83dac0364ae7,kay,https://play-lh.googleusercontent.com/a-/ALV-U...,It's surprising how bad the app is compared to...,1,146,1.0.0026,2023-08-06 01:47:00,None,NaT,1.0.0026,most_relevant,com.openai.chatgpt
4,e8a2aa7d-46fc-4975-a1a4-d9c9da78d244,R Meowing,https://play-lh.googleusercontent.com/a-/ALV-U...,It looks cool but it just doesn't work on my p...,1,12,1.2023.263,2023-10-04 22:08:44,None,NaT,1.2023.263,most_relevant,com.openai.chatgpt
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55934,31d4eae4-2060-48ef-b742-613169f3cdb8,Maruf Ahmed Shikder,https://play-lh.googleusercontent.com/a/ACg8oc...,অবশেষে chat GPT play store এ এসেছে🥰 । এতে আমি ...,5,0,1.0.0016,2023-07-26 01:24:49,None,NaT,1.0.0016,newest,com.openai.chatgpt
55935,04efe9c4-db6c-40d8-98e3-b55d23875b6d,Samir Sheikh,https://play-lh.googleusercontent.com/a-/ALV-U...,"App bahut acha hai, lakin isme kuch puchne par...",5,0,1.0.0016,2023-07-25 21:20:28,None,NaT,1.0.0016,newest,com.openai.chatgpt
55936,2d5d7293-9ec4-496c-8589-31dcf01cc85c,Vivek Verma,https://play-lh.googleusercontent.com/a-/ALV-U...,This is the best app. बढिया है और एक ही है ❤,5,0,1.0.0016,2023-07-25 23:25:51,None,NaT,1.0.0016,newest,com.openai.chatgpt
55937,254b062b-934a-45a1-955e-06b340f529d7,Kamal 315,https://play-lh.googleusercontent.com/a-/ALV-U...,First downloader iski video mein apne YouTube ...,5,0,1.0.0016,2023-07-25 20:47:43,None,NaT,1.0.0016,newest,com.openai.chatgpt


In [13]:
app_reviews_df.to_csv('Input.csv')